# Implementing logistic regression from scratch

The goal of this notebook is to implement your own logistic regression classifier. You will:

 * Extract features from Amazon product reviews.
 * Convert an DataFrame into a NumPy array.
 * Implement the link function for logistic regression.
 * Write a function to compute the derivative of the log likelihood function with respect to a single coefficient.
 * Implement gradient ascent.
 * Given a set of coefficients, predict sentiments.
 * Compute classification accuracy for the logistic regression model.
 
Let's get started!
    

## Fire up Scikit-learn, Pandas and Numpy

In [4]:
import pandas as pd
import numpy as np
from sklearn import datasets, linear_model

## Load review dataset

For this assignment, we will use a subset of the Amazon product review dataset. The subset was chosen to contain similar numbers of positive and negative reviews, as the original dataset consisted primarily of positive reviews.

In [5]:
products = pd.read_csv('amazon_baby_subset.csv')

One column of this dataset is 'sentiment', corresponding to the class label with +1 indicating a review with positive sentiment and -1 indicating one with negative sentiment.

In [6]:
products['sentiment'].head()

0    1
1    1
2    1
3    1
4    1
Name: sentiment, dtype: int64

Let us quickly explore more of this dataset.  The 'name' column indicates the name of the product.  Here we list the first 10 products in the dataset.  We then count the number of positive and negative reviews.

In [7]:
products.head(10)['name']

0    Stop Pacifier Sucking without tears with Thumb...
1      Nature's Lullabies Second Year Sticker Calendar
2      Nature's Lullabies Second Year Sticker Calendar
3                          Lamaze Peekaboo, I Love You
4    SoftPlay Peek-A-Boo Where's Elmo A Children's ...
5                            Our Baby Girl Memory Book
6    Hunnt&reg; Falling Flowers and Birds Kids Nurs...
7    Blessed By Pope Benedict XVI Divine Mercy Full...
8    Cloth Diaper Pins Stainless Steel Traditional ...
9    Cloth Diaper Pins Stainless Steel Traditional ...
Name: name, dtype: object

In [8]:
print ('# of positive reviews =', len(products[products['sentiment']==1]))
print ('# of negative reviews =', len(products[products['sentiment']==-1]))

# of positive reviews = 26579
# of negative reviews = 26493


**Note:** For this assignment, we eliminated class imbalance by choosing 
a subset of the data with a similar number of positive and negative reviews. 

## Apply text cleaning on the review data

In this section, we will perform some simple feature cleaning using **Pandas**. The last assignment used all words in building bag-of-words features, but here we limit ourselves to 193 words (for simplicity). We compiled a list of 193 most frequent words into a JSON file. 

Now, we will load these words from this JSON file:

In [9]:
import json
with open('important_words.json', 'r') as f: # Reads the list of most frequent words
    important_words = json.load(f)
important_words = [str(s) for s in important_words]

In [10]:
print (important_words)

['baby', 'one', 'great', 'love', 'use', 'would', 'like', 'easy', 'little', 'seat', 'old', 'well', 'get', 'also', 'really', 'son', 'time', 'bought', 'product', 'good', 'daughter', 'much', 'loves', 'stroller', 'put', 'months', 'car', 'still', 'back', 'used', 'recommend', 'first', 'even', 'perfect', 'nice', 'bag', 'two', 'using', 'got', 'fit', 'around', 'diaper', 'enough', 'month', 'price', 'go', 'could', 'soft', 'since', 'buy', 'room', 'works', 'made', 'child', 'keep', 'size', 'small', 'need', 'year', 'big', 'make', 'take', 'easily', 'think', 'crib', 'clean', 'way', 'quality', 'thing', 'better', 'without', 'set', 'new', 'every', 'cute', 'best', 'bottles', 'work', 'purchased', 'right', 'lot', 'side', 'happy', 'comfortable', 'toy', 'able', 'kids', 'bit', 'night', 'long', 'fits', 'see', 'us', 'another', 'play', 'day', 'money', 'monitor', 'tried', 'thought', 'never', 'item', 'hard', 'plastic', 'however', 'disappointed', 'reviews', 'something', 'going', 'pump', 'bottle', 'cup', 'waste', 'retu

Now, we will perform 2 simple data transformations:

1. Remove punctuation using [Python's built-in](https://docs.python.org/2/library/string.html) string functionality.
2. Compute word counts (only for **important_words**)

We start with *Step 1* which can be done as follows:

In [11]:
products = products.fillna({'review':''})  # fill in N/A's in the review column

In [12]:
def remove_punctuation(text):
    import string
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator) 

In [13]:
products['review_clean'] = products['review'].astype(str).apply(remove_punctuation) #astype(str) makes sure all reviews are strings

Now we proceed with *Step 2*. For each word in **important_words**, we compute a count for the number of times the word occurs in the review. We will store this count in a separate column (one for each word). The result of this feature processing is a single column for each word in **important_words** which keeps a count of the number of times the respective word occurs in the review text.


**Note:** There are several ways of doing this. In this assignment, we use the built-in *count* function for Python lists. Each review string is first split into individual words and the number of occurances of a given word is counted.

In [14]:
for word in important_words:
    products[word] = products['review_clean'].apply(lambda s : s.split().count(word))

6.. The data frame products should contain one column for each of the 193 important_words. As an example, the column perfect contains a count of the number of times the word perfect occurs in each of the reviews.

7.. Now, write some code to compute the number of product reviews that contain the word perfect.

In [15]:
len(products['perfect'][products['perfect'] !=0])

2955

Quiz Question. How many reviews contain the word perfect?

In [16]:
print('The answer is:', len(products['perfect'][products['perfect'] !=0]))

The answer is: 2955


## Convert DataFrame to NumPy array

8.. It is now time to convert our data frame to a multi-dimensional array. Look for a package that provides a highly optimized matrix operations. In the case of Python, NumPy is a good choice.

Write a function that extracts columns from a data frame and converts them into a multi-dimensional array. We plan to use them throughout the course, so make sure to get this function right.

The function should accept three parameters:

dataframe: a data frame to be converted
features: a list of string, containing the names of the columns that are used as features.
label: a string, containing the name of the single column that is used as class labels.
The function should return two values:

one 2D array for features
one 1D array for class labels
The function should do the following:

Prepend a new column constant to dataframe and fill it with 1's. This column takes account of the intercept term. Make sure that the constant column appears first in the data frame.
Prepend a string 'constant' to the list features. Make sure the string 'constant' appears first in the list.
Extract columns in dataframe whose names appear in the list features.
Convert the extracted columns into a 2D array using a function in the data frame library. If you are using Pandas, you would use as_matrix() function.
Extract the single column in dataframe whose name corresponds to the string label.
Convert the column into a 1D array.
Return the 2D array and the 1D array.

In [17]:
def get_numpy_data(dataframe, features, label):
    dataframe['constant'] = 1
    features = ['constant'] + features
    #print(features)
    features_frame = dataframe[features]
    #print(dataframe[features])
    #print(features_frame)
    feature_matrix = features_frame.values
    label_sarray = dataframe[label]
    label_array = label_sarray.values
    return(feature_matrix, label_array)

9.. Using the function written in #8, extract two arrays feature_matrix and sentiment. The 2D array feature_matrix would contain the content of the columns given by the list important_words. The 1D array sentiment would contain the content of the column sentiment.

Let us convert the data into NumPy arrays.

In [18]:
# Warning: This may take a few minutes...
feature_matrix, sentiment = get_numpy_data(products, important_words, 'sentiment') 

In [19]:
feature_matrix.shape

(53072, 194)

** Quiz Question:** How many features are there in the **feature_matrix**?

** Quiz Question:** Assuming that the intercept is present, how does the number of features in **feature_matrix** relate to the number of features in the logistic regression model?

In [20]:
print('The answer is 194 features are there in the feature_matrix')

The answer is 194 features are there in the feature_matrix


In [167]:
print('The answer is: Assuming that the intercept is present, the number of features in feature_matrix is the same as the number of features in the logistic regression model')

The answer is: Assuming that the intercept is present, the number of features in feature_matrix is the same as the number of features in the logistic regression model


Now, let us see what the **sentiment** column looks like:

In [22]:
sentiment

array([ 1,  1,  1, ..., -1, -1, -1])

## Estimating conditional probability with link function

Recall from lecture that the link function is given by:
$$
P(y_i = +1 | \mathbf{x}_i,\mathbf{w}) = \frac{1}{1 + \exp(-\mathbf{w}^T h(\mathbf{x}_i))},
$$

where the feature vector $h(\mathbf{x}_i)$ represents the word counts of **important_words** in the review  $\mathbf{x}_i$. Complete the following function that implements the link function:

~Take two parameters: feature_matrix and coefficients.

~First compute the dot product of feature_matrix and coefficients.

~Then compute the link function 

~Return the predictions given by the link function.

Your code should be analogous to the following Python function:

In [23]:
'''
produces probablistic estimate for P(y_i = +1 | x_i, w).
estimate ranges between 0 and 1.
'''
def predict_probability(feature_matrix, coefficients):
    # Take dot product of feature_matrix and coefficients  
    # YOUR CODE HERE
    score = np.dot(feature_matrix, coefficients)
    
    # Compute P(y_i = +1 | x_i, w) using the link function
    # YOUR CODE HERE
    predictions = 1/(1+(np.exp(-score)))
    
    # return predictions
    return predictions

**Aside**. How the link function works with matrix algebra

Since the word counts are stored as columns in **feature_matrix**, each $i$-th row of the matrix corresponds to the feature vector $h(\mathbf{x}_i)$:
$$
[\text{feature_matrix}] =
\left[
\begin{array}{c}
h(\mathbf{x}_1)^T \\
h(\mathbf{x}_2)^T \\
\vdots \\
h(\mathbf{x}_N)^T
\end{array}
\right] =
\left[
\begin{array}{cccc}
h_0(\mathbf{x}_1) & h_1(\mathbf{x}_1) & \cdots & h_D(\mathbf{x}_1) \\
h_0(\mathbf{x}_2) & h_1(\mathbf{x}_2) & \cdots & h_D(\mathbf{x}_2) \\
\vdots & \vdots & \ddots & \vdots \\
h_0(\mathbf{x}_N) & h_1(\mathbf{x}_N) & \cdots & h_D(\mathbf{x}_N)
\end{array}
\right]
$$

By the rules of matrix multiplication, the score vector containing elements $\mathbf{w}^T h(\mathbf{x}_i)$ is obtained by multiplying **feature_matrix** and the coefficient vector $\mathbf{w}$.
$$
[\text{score}] =
[\text{feature_matrix}]\mathbf{w} =
\left[
\begin{array}{c}
h(\mathbf{x}_1)^T \\
h(\mathbf{x}_2)^T \\
\vdots \\
h(\mathbf{x}_N)^T
\end{array}
\right]
\mathbf{w}
= \left[
\begin{array}{c}
h(\mathbf{x}_1)^T\mathbf{w} \\
h(\mathbf{x}_2)^T\mathbf{w} \\
\vdots \\
h(\mathbf{x}_N)^T\mathbf{w}
\end{array}
\right]
= \left[
\begin{array}{c}
\mathbf{w}^T h(\mathbf{x}_1) \\
\mathbf{w}^T h(\mathbf{x}_2) \\
\vdots \\
\mathbf{w}^T h(\mathbf{x}_N)
\end{array}
\right]
$$

**Checkpoint**

Just to make sure you are on the right track, we have provided a few examples. If your `predict_probability` function is implemented correctly, then the outputs will match:

In [24]:
dummy_feature_matrix = np.array([[1.,2.,3.], [1.,-1.,-1]])
dummy_coefficients = np.array([1., 3., -1.])

correct_scores      = np.array( [ 1.*1. + 2.*3. + 3.*(-1.),          1.*1. + (-1.)*3. + (-1.)*(-1.) ] )
correct_predictions = np.array( [ 1./(1+np.exp(-correct_scores[0])), 1./(1+np.exp(-correct_scores[1])) ] )

print ('The following outputs must match ')
print ('------------------------------------------------')
print ('correct_predictions           =', correct_predictions)
print ('output of predict_probability =', predict_probability(dummy_feature_matrix, dummy_coefficients))

The following outputs must match 
------------------------------------------------
correct_predictions           = [ 0.98201379  0.26894142]
output of predict_probability = [ 0.98201379  0.26894142]


## Compute derivative of log likelihood with respect to a single coefficient

Recall from lecture:
$$
\frac{\partial\ell}{\partial w_j} = \sum_{i=1}^N h_j(\mathbf{x}_i)\left(\mathbf{1}[y_i = +1] - P(y_i = +1 | \mathbf{x}_i, \mathbf{w})\right)
$$

We will now write a function that computes the derivative of log likelihood with respect to a single coefficient $w_j$. The function accepts two arguments:
* `errors` vector containing $\mathbf{1}[y_i = +1] - P(y_i = +1 | \mathbf{x}_i, \mathbf{w})$ for all $i$.
* `feature` vector containing $h_j(\mathbf{x}_i)$  for all $i$. 

Complete the following code block:

This corresponds to the j-th column of feature_matrix.

The function should do the following:

~Take two parameters errors and feature.

~Compute the dot product of errors and feature.

~Return the dot product. This is the derivative with respect to a single coefficient w_j.

Your code should be analogous to the following Python function:

In [50]:
def feature_derivative(errors, feature):     
    # Compute the dot product of errors and feature
    derivative = np.dot(errors, feature)
        # Return the derivative
    return derivative

In the main lecture, our focus was on the likelihood.  In the advanced optional video, however, we introduced a transformation of this likelihood---called the log likelihood---that simplifies the derivation of the gradient and is more numerically stable.  Due to its numerical stability, we will use the log likelihood instead of the likelihood to assess the algorithm.

The log likelihood is computed using the following formula (see the advanced optional video if you are curious about the derivation of this equation):

$$\ell\ell(\mathbf{w}) = \sum_{i=1}^N \Big( (\mathbf{1}[y_i = +1] - 1)\mathbf{w}^T h(\mathbf{x}_i) - \ln\left(1 + \exp(-\mathbf{w}^T h(\mathbf{x}_i))\right) \Big) $$

We provide a function to compute the log likelihood for the entire dataset. 

In [51]:
def compute_log_likelihood(feature_matrix, sentiment, coefficients):
    indicator = (sentiment==+1)
    scores = np.dot(feature_matrix, coefficients)
    lp = np.sum((indicator-1)*scores - np.log(1. + np.exp(-scores)))
    return lp

**Checkpoint**

Just to make sure we are on the same page, run the following code block and check that the outputs match.

In [52]:
dummy_feature_matrix = np.array([[1.,2.,3.], [1.,-1.,-1]])
dummy_coefficients = np.array([1., 3., -1.])
dummy_sentiment = np.array([-1, 1])

correct_indicators  = np.array( [ -1==+1,                                       1==+1 ] )
correct_scores      = np.array( [ 1.*1. + 2.*3. + 3.*(-1.),                     1.*1. + (-1.)*3. + (-1.)*(-1.) ] )
correct_first_term  = np.array( [ (correct_indicators[0]-1)*correct_scores[0],  (correct_indicators[1]-1)*correct_scores[1] ] )
correct_second_term = np.array( [ np.log(1. + np.exp(-correct_scores[0])),      np.log(1. + np.exp(-correct_scores[1])) ] )

correct_ll          =      sum( [ correct_first_term[0]-correct_second_term[0], correct_first_term[1]-correct_second_term[1] ] ) 

print ('The following outputs must match ')
print ('------------------------------------------------')
print ('correct_log_likelihood           =', correct_ll)
print ('output of compute_log_likelihood =', compute_log_likelihood(dummy_feature_matrix, dummy_sentiment, dummy_coefficients))

The following outputs must match 
------------------------------------------------
correct_log_likelihood           = -5.33141161544
output of compute_log_likelihood = -5.33141161544


## Taking gradient steps

Now we are ready to implement our own logistic regression. All we have to do is to write a gradient ascent function that takes gradient steps towards the optimum. 

Complete the following function to solve the logistic regression model using gradient ascent:

The function carries out the following steps:

~Initialize vector coefficients to initial_coefficients.

~Predict the class probability using your predict_probability function and save it to variable predictions.

~Compute indicator value for (by comparing sentiment against +1. Save it to variable indicator.

~Compute the errors as difference between indicator and predictions. Save the errors to variable errors.
                              
~For each j-th coefficient, compute the per-coefficient derivative by calling feature_derivative with the j-th column of feature_matrix. Then increment the j-th coefficient by (step_size*derivative).

~Once in a while, insert code to print out the log likelihood.
                              
~Repeat steps 2-6 for max_iter times.
                              
At the end of day, your code should be analogous to the following Python function (with blanks filled in):

In [75]:
from math import sqrt
def logistic_regression(feature_matrix, sentiment, initial_coefficients, step_size, max_iter):
    coefficients = np.array(initial_coefficients) # make sure it's a numpy array
    for itr in range(max_iter):
        # Predict P(y_i = +1|x_1,w) using your predict_probability() function
        # YOUR CODE HERE
        #print('iteration',itr)
        predictions = predict_probability(feature_matrix, coefficients)
        #print('predictions', predictions)

        # Compute indicator value for (y_i = +1)
        indicator = (sentiment==+1)
        #print('indicator',indicator)

        # Compute the errors as indicator - predictions
        errors = indicator - predictions
        #print(errors)
        #print(errors.shape)

        for j in range(len(coefficients)): # loop over each coefficient
            # Recall that feature_matrix[:,j] is the feature column associated with coefficients[j]
            # compute the derivative for coefficients[j]. Save it in a variable called derivative
            # YOUR CODE HERE
            #print('coefficient',j)
            derivative = feature_derivative(errors, feature_matrix[:,j])
            #print('derivative',derivative)

            # add the step size times the derivative to the current coefficient
            # YOUR CODE HERE
            coefficients[j] = coefficients[j] + step_size*derivative
        #print('coefficients',coefficients)
        #print('coefficients len',len(coefficients))

        # Checking whether log likelihood is increasing
        if itr <= 15 or (itr <= 100 and itr % 10 == 0) or (itr <= 1000 and itr % 100 == 0) \
        or (itr <= 10000 and itr % 1000 == 0) or itr % 10000 == 0:
            lp = compute_log_likelihood(feature_matrix, sentiment, coefficients)
            print ('iteration %*d: log likelihood of observed labels = %.8f' % \
                (int(np.ceil(np.log10(max_iter))), itr, lp))
    return coefficients

Now, let us run the logistic regression solver.

In [76]:
coefficients = logistic_regression(feature_matrix, sentiment, initial_coefficients=np.zeros(194),
                                   step_size=1e-7, max_iter=301)

iteration   0: log likelihood of observed labels = -36780.91768478
iteration   1: log likelihood of observed labels = -36775.13434712
iteration   2: log likelihood of observed labels = -36769.35713564
iteration   3: log likelihood of observed labels = -36763.58603240
iteration   4: log likelihood of observed labels = -36757.82101962
iteration   5: log likelihood of observed labels = -36752.06207964
iteration   6: log likelihood of observed labels = -36746.30919497
iteration   7: log likelihood of observed labels = -36740.56234821
iteration   8: log likelihood of observed labels = -36734.82152213
iteration   9: log likelihood of observed labels = -36729.08669961
iteration  10: log likelihood of observed labels = -36723.35786366
iteration  11: log likelihood of observed labels = -36717.63499744
iteration  12: log likelihood of observed labels = -36711.91808422
iteration  13: log likelihood of observed labels = -36706.20710739
iteration  14: log likelihood of observed labels = -36700.5020

In [83]:
coefficients

array([  5.16220157e-03,   1.55656966e-02,  -8.50204675e-03,
         6.65460842e-02,   6.58907629e-02,   5.01743882e-03,
        -5.38601484e-02,  -3.50488413e-03,   6.47945868e-02,
         4.54356263e-02,   3.98353364e-03,   2.00775410e-02,
         3.01350011e-02,  -2.87115530e-02,   1.52161964e-02,
         2.72592062e-04,   1.19448177e-02,  -1.82461935e-02,
        -1.21706420e-02,  -4.15110334e-02,   2.76820391e-03,
         1.77031999e-02,  -4.39700067e-03,   4.49764014e-02,
         9.90916464e-03,   8.99239081e-04,  -1.36219516e-03,
         1.26859357e-02,   8.26466695e-03,  -2.77426972e-02,
         6.10128809e-04,   1.54084501e-02,  -1.32134753e-02,
        -3.00512492e-02,   2.97399371e-02,   1.84087080e-02,
         2.86178752e-03,  -1.05768015e-02,  -6.57350362e-04,
        -1.01476555e-02,  -4.79579528e-03,   7.50891810e-03,
         4.27938289e-03,   3.06785501e-03,  -2.20317661e-03,
         9.57273354e-03,   9.91666827e-05,  -1.98462567e-02,
         1.75702722e-02,

**Quiz Question:** As each iteration of gradient ascent passes, does the log likelihood increase or decrease?

In [46]:
print('The answer is: increase')

The answer is: increase


## Predicting sentiments

Recall from lecture that class predictions for a data point $\mathbf{x}$ can be computed from the coefficients $\mathbf{w}$ using the following formula:
$$
\hat{y}_i = 
\left\{
\begin{array}{ll}
      +1 & \mathbf{x}_i^T\mathbf{w} > 0 \\
      -1 & \mathbf{x}_i^T\mathbf{w} \leq 0 \\
\end{array} 
\right.
$$

Now, we will write some code to compute class predictions. We will do this in two steps:
* **Step 1**: First compute the **scores** using **feature_matrix** and **coefficients** using a dot product.
* **Step 2**: Using the formula above, compute the class predictions from the scores.

Step 1 can be implemented as follows:

In [85]:
# Compute the scores as a dot product between feature_matrix and coefficients.
scores = np.dot(feature_matrix, coefficients)
scores

array([ 0.05104571, -0.02936473,  0.02411584, ..., -0.40986295,
        0.01411436, -0.06755923])

In [107]:
len(scores)

53072

Now, complete the following code block for **Step 2** to compute the class predictions using the **scores** obtained above:

In [102]:
def predict_sentiments(score):
    if score>0:
        predicted_label = +1
    else:
        predicted_label = -1
    return predicted_label
    

In [103]:
predict_sentiments(scores[0])

1

** Quiz Question: ** How many reviews were predicted to have positive sentiment?

In [118]:
predicted_labels = []
for i in range(len(scores)):
    predict_sentiments(scores[i])
    #print(predict_sentiments(scores[i]))
    predicted_labels.append(predict_sentiments(scores[i]))

In [119]:
predicted_labels_df = pd.DataFrame({'predicted_labels':predicted_labels})
predicted_labels_df

,predicted_labels
0,1
1,-1
2,1
3,1
4,1
5,1
6,1
7,1
8,1
9,-1


Quiz question: How many reviews were predicted to have positive sentiment?

In [123]:
print('The answer is:',len(predicted_labels_df[predicted_labels_df['predicted_labels']>0]))

The answer is: 25126


## Measuring accuracy

We will now measure the classification accuracy of the model. Recall from the lecture that the classification accuracy can be computed as follows:

$$
\mbox{accuracy} = \frac{\mbox{# correctly classified data points}}{\mbox{# total data points}}
$$

Complete the following code block to compute the accuracy of the model.

In [127]:
predicted_sentiment= predicted_labels_df.values.flatten()
predicted_sentiment

array([ 1, -1,  1, ..., -1,  1, -1])

In [128]:
predicted_sentiment.shape

(53072,)

In [130]:
sentiment

array([ 1,  1,  1, ..., -1, -1, -1])

In [132]:
diff = predicted_sentiment-sentiment
diff

array([ 0, -2,  0, ...,  0,  2,  0])

In [136]:
len(diff)

53072

In [143]:
diff_num_zeros = np.sum(diff == 0)
diff_num_zeros

39903

In [144]:
num_mistakes = len(diff)-diff_num_zeros # YOUR CODE HERE
num_correctness = np.sum(diff == 0)
accuracy = num_correctness/len(diff) # YOUR CODE HERE
print ("-----------------------------------------------------")
print ('# Reviews   correctly classified =', num_correctness)
print ('# Reviews incorrectly classified =', num_mistakes)
print ('# Reviews total                  =', len(diff))
print ("-----------------------------------------------------")
print ('Accuracy = %.2f' % accuracy)

-----------------------------------------------------
# Reviews   correctly classified = 39903
# Reviews incorrectly classified = 13169
# Reviews total                  = 53072
-----------------------------------------------------
Accuracy = 0.75


Quiz question: What is the accuracy of the model on predictions made above? (round to 2 digits of accuracy)

In [147]:
print('The answer is:',np.round(accuracy,decimals=2))

The answer is: 0.75


## Which words contribute most to positive & negative sentiments?

Recall that in Module 2 assignment, we were able to compute the "**most positive words**". These are words that correspond most strongly with positive reviews. In order to do this, we will first do the following:
* Treat each coefficient as a tuple, i.e. (**word**, **coefficient_value**).
* Sort all the (**word**, **coefficient_value**) tuples by **coefficient_value** in descending order.

In [151]:
coefficients = list(coefficients[1:]) # exclude intercept
word_coefficient_tuples = [(word, coefficient) for word, coefficient in zip(important_words, coefficients)]
word_coefficient_tuples = sorted(word_coefficient_tuples, key=lambda x:x[1], reverse=True)

Now, **word_coefficient_tuples** contains a sorted list of (**word**, **coefficient_value**) tuples. The first 10 elements in this list correspond to the words that are most positive.

### Ten "most positive" words

Now, we compute the 10 words that have the most positive coefficient values. These words are associated with positive sentiment.

In [156]:
word_coefficient_tuples[0:10]

[('great', 0.066546084170457695),
 ('love', 0.065890762922123258),
 ('easy', 0.06479458680257838),
 ('little', 0.045435626308421365),
 ('loves', 0.044976401394906038),
 ('well', 0.030135001092107077),
 ('perfect', 0.029739937104968462),
 ('old', 0.020077541034775378),
 ('nice', 0.018408707995268992),
 ('daughter', 0.017703199905701694)]

** Quiz Question:** Which word is **not** present in the top 10 "most positive" words?

### Ten "most negative" words

Next, we repeat this exercise on the 10 most negative words.  That is, we compute the 10 words that have the most negative coefficient values. These words are associated with negative sentiment.

In [159]:
word_coefficient_tuples_2 = sorted(word_coefficient_tuples, key=lambda x:x[1], reverse=False)
word_coefficient_tuples_2

[('would', -0.053860148445203142),
 ('product', -0.041511033392108904),
 ('money', -0.038982037286487116),
 ('work', -0.03306951529475273),
 ('even', -0.030051249236035804),
 ('disappointed', -0.028978976142317068),
 ('get', -0.028711552980192574),
 ('back', -0.027742697230661334),
 ('return', -0.026592778462247283),
 ('monitor', -0.024482100545891717),
 ('waste', -0.02404274807115496),
 ('thought', -0.021394348543682485),
 ('could', -0.01984625666077721),
 ('tried', -0.018702371424325834),
 ('way', -0.018359460662945686),
 ('time', -0.018246193486087033),
 ('buy', -0.017737543997218049),
 ('item', -0.01713786701085479),
 ('returned', -0.016001798500102516),
 ('broke', -0.01553769895565389),
 ('better', -0.015040658977043391),
 ('plastic', -0.014977044903587941),
 ('reviews', -0.014866319449976982),
 ('cheap', -0.014709833465080665),
 ('thing', -0.013868727265297583),
 ('hard', -0.013756676731261401),
 ('first', -0.013213475301677043),
 ('never', -0.01296904654631929),
 ('something', -

In [160]:
word_coefficient_tuples_2[0:10]

[('would', -0.053860148445203142),
 ('product', -0.041511033392108904),
 ('money', -0.038982037286487116),
 ('work', -0.03306951529475273),
 ('even', -0.030051249236035804),
 ('disappointed', -0.028978976142317068),
 ('get', -0.028711552980192574),
 ('back', -0.027742697230661334),
 ('return', -0.026592778462247283),
 ('monitor', -0.024482100545891717)]

** Quiz Question:** Which word is **not** present in the top 10 "most negative" words?